In [ ]:
# Mount a google drive with the person_detection repository inside from here https://github.com/Yannick947/person_detection

In [2]:
!pip install gdown==3.6.4
!pip install tensorflow-gpu==2.2.0 --user
!pip install keras==2.3.1
!pip install efficientnet==1.1.0

     |████████████████████████████████| 516.2MB 29kB/s 
     |████████████████████████████████| 460kB 56kB/s 
     |████████████████████████████████| 3.0MB 46.9MB/s 
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.2.0 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
     |████████████████████████████████| 378kB 6.3MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [3]:
%cd ../../../../content
!git clone https://github.com/Yannick947/keras-retinanet.git --branch master
%cd keras-retinanet/
!pip install .
!python setup.py build_ext --inplace
%cd ../sample_data/
!git clone https://github.com/Yannick947/pcds_images.git
%cd ../../

/content
Cloning into 'keras-retinanet'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 6619 (delta 74), reused 80 (delta 34), pack-reused 6490
Receiving objects: 100% (6619/6619), 55.97 MiB | 19.89 MiB/s, done.
Resolving deltas: 100% (4036/4036), done.
/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=184380 sha256=feaad3c67958ae0f9ce217c9d9ffdc85719728341a8155bd280416c9bbb8e3e7
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13345 sha256=a0d965d1a23bedc7c2d0d63f377a0cd71899e5f4f097c3a6aed9ef5dc0500b97
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built 

In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from time import gmtime, strftime

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

annot_train_path = '/content/keras-retinanet/annotations/annot_train_pcds.csv'
annot_test_path = '/content/keras-retinanet/annotations/annot_test_pcds.csv'
classes_path = '/content/keras-retinanet/annotations/classes_filtered.csv'
HEADER= ['image_name', 'x1', 'y1', 'x2', 'y2', 'label']
df_train = pd.read_csv(annot_train_path, names=HEADER)
num_train_images = len(df_train.image_name.unique())
print('Number of training images: ', num_train_images)

Number of training images:  486


# Training



In [5]:
#Paths to pretrained COCO weights
PRETRAINED_RES50_VANILLA = '/content/drive/My\ Drive/person_detection/keras-retinanet/snapshots/resnet50_vanilla.h5'
PRETRAINED_RES152_VANILLA = '' # Download official weights if desired
# Path to save snapshots
SNAPSHOT_PATH = '/content/drive/My\ Drive/person_detection/keras-retinanet/snapshots_pcds/'
#resnet50, map69, standard resize, no augmentation, lr 1e-5
PRETRAINED_MODEL_WI = '/content/drive/My\ Drive/person_detection/keras-retinanet/snapshots_pcds/22-05-2020_071626_resnet50_20_mAP69.h5'

In [7]:
%cd /content/keras-retinanet/
for _ in range(10):

  logdir = '/content/drive/My\ Drive/person_detection/keras-retinanet/tensorboard_pcds/{}_2'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
  aug = np.random.uniform(low=0.0, high=1.5)
  vis_aug = np.random.uniform(low=0.0, high=2.0)
  lr = np.random.uniform(low=3e-4, high=5e-6)

  print('Training for following parameters: \nVisual augmentation factor: {} \nAugmentation factor: {}, \nlr: {}'.format(vis_aug, aug, lr))
  !python keras_retinanet/bin/train.py  --weights {PRETRAINED_RES50_VANILLA}\
                                        --lr $lr\
                                        --batch-size 2\
                                        --backbone 'resnet50'\
                                        --steps 500\
                                        --epochs 12 \
                                        --weighted-average\
                                        --random-transform\
                                        --visual-aug-factor $vis_aug\
                                        --augmentation-factor $aug\
                                        --tensorboard-dir {logdir}\
                                        --compute-val-loss\
                                        --workers 0\
                                        --no-snapshots\
                                        --multiprocessing\
                                        --max-queue-size 16\
                                        --num-train-imgs 500\
                                        csv /content/keras-retinanet/annotations/annot_train_pcds.csv /content/keras-retinanet/annotations/classes.csv\
                                        --val-annotations /content/keras-retinanet/annotations/annot_test_pcds.csv\

%cd ../../../../../

/content/keras-retinanet
Training for following parameters: 
Visual augmentation factor: 1.3196258283463558 
Augmentation factor: 0.31779810421562216, 
lr: 7.165848883637022e-05
Using TensorFlow backend.
/content/keras-retinanet/annotations/annot_test_pcds.csv
Creating model, this may take a second...
2020-10-24 09:53:29.163039: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-24 09:53:29.220111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-24 09:53:29.221001: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-10-24 09:53:29.221311: I tensorflow/stream_

### Anchor Optimization

In [ ]:
%cd /content/drive/My Drive/person_detection
!git clone https://github.com/martinzlocha/anchor-optimization.git
%cd anchor-optimization
!pip install .
!python setup.py build_ext --inplace

In [ ]:
!pip install tensorflow-gpu==1.15
!python ./../anchor-optimization/optimize_anchors.py ./annot_train_filtered_600_2500.csv
!pip uninstall tensorflow-gpu==1.15